# Solving the problem of 3-coloring using ILP

This is a small example to show how you can encode the problem of [3-coloring](https://en.wikipedia.org/wiki/Graph_coloring#Vertex_coloring) into ILP and using an ILP solver to solve the problem.

We will use the the [Gurobi Optimizer](https://www.gurobi.com/) in Python for this (see this [example](ilp.ipynb) for details on how to set this up). Let's start by importing some relevant things, and by creating an ILP model.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model();

## 3-coloring

In the problem of 3-coloring, you are given as input a finite (undirected) [graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)) $G = (V,E)$, consisting of a set $V$ of *vertices* (or *nodes*) and a set $E$ of *edges*. Each edge $e \in E$ consists of a set $\{v_1,v_2\}$ of exactly two vertices in $V$.

A *coloring* of a graph $G = (V,E)$ is a function $\mu : V \rightarrow C$ that assigns to each vertex $v \in V$ a color $\mu(v) \in C$. The coloring $\mu$ is called *proper* if for every edge $\{v_1,v_2\} \in E$, the coloring assigns different colors to the two endpoints of the edge—that is, $\mu(v_1) \neq \mu(v_2)$.

In the problem of 3-coloring, the question is to decide if there exists a proper coloring $\mu : V \rightarrow \{1,2,3\}$ of the graph that only uses three colors.

## Encoding the graph and colorings

Take some graph $G = (V,E)$, that comprises an input for the 3-coloring problem.
Suppose that the vertices $V = \{ v_1,\dotsc,v_n \}$ are numbered from $1$ to $n$.

We will encode this graph in Python in the following way.

To explain this, take the following example graph $G = (V,E)$, where $V = \{v_1,v_2,v_3,v_4\}$ and $E = \{ \{v_1,v_2\}, \{v_1,v_3\}, \{v_2,v_3\}, \{v_2,v_4\}, \{v_3,v_4\} \}$. We encode the set of edges as a list of pairs, where each vertex is indicated by its index. 

In [2]:
num_vertices = 4;
edges = [(1,2),(1,3),(2,3),(2,4),(3,4)];

Then, we will have to represent the possible 3-colorings of the graph $G$ using integer variables. We will do this as follows. For each vertex $v_i$ and each color $c$, we will have a binary variable $x_{i,c}$ that indicates whether $v_i$ gets colored with color $c$. So it will get value $1$ if $v_i$ gets colored with $c$, and value $0$ otherwise.

In [3]:
num_colors = 3;
vars = dict()
for i in range(1,num_vertices+1):
    for c in range(1,num_colors+1):
        vars[(i,c)] = model.addVar(vtype=GRB.BINARY, name="x({},{})".format(i,c));

Then, to ensure that the value given to these binary variables corresponds to a 3-coloring of the graph, we will need to add some linear constraints.

We begin by adding, for each vertex $v_i$, a constraint that states that there must be exactly one $c$ for which $x_{i,c}$ gets value $1$.
For each $v_i$, we add the constraint: $x_{i,1} + x_{i,2} + x_{i,3} = 1$.

In [4]:
for i in range(1,num_vertices+1):
    model.addConstr(gp.quicksum([vars[(i,c)] for c in range(1,num_colors+1)]) == 1);

These constraints ensure that the assignments to the integer variables satisfying these constraints correspond exactly to all possible 3-colorings of the vertices $v_1,\dotsc,v_4$.

## Ensuring that colorings are proper

To make sure that only assignments are allowed that correspond to *proper* 3-colorings, we need to add some further constraints.

For each edge $\{ v_{i_1},v_{i_2} \} \in E$, and each color $c$, we add a constraint that states that the vertices $v_{i_1}$ and $v_{i_2}$ do not both get colored with color $c$.
For each edge $\{ v_{i_1},v_{i_2} \} \in E$, we add the constraints: $x_{i_1,1} + x_{i_2,1} \leq 1$, $x_{i_1,2} + x_{i_2,2} \leq 1$, $x_{i_1,3} + x_{i_2,3} \leq 1$.

In [5]:
for (i1,i2) in edges:
    for c in range(1,num_colors+1):
        model.addConstr(vars[(i1,c)] + vars[(i2,c)] <= 1);

## Calling the ILP solver and constructing a coloring

All together, the constraints that we added ensure the following. Every assignment that satisfies these constraints (if such an assignment exists) corresponds to a proper 3-coloring of the graph $G$. Therefore, we can now call the ILP solver to find a solution to the ILP problem that we constructed, if it exists. And if it exists, we can use it to construct a proper 3-coloring of the graph.

For our simple example, we can satisfy all the linear constraints, and we get a proper 3-coloring of the graph.

In [ ]:
model.optimize();

In [7]:
if model.status == GRB.OPTIMAL:
    print("The graph is 3-colorable:");
    for i in range(1,num_vertices+1):
        for c in range(1,num_colors+1):
            if vars[(i,c)].x == 1:
                print("- Vertex {} gets color {}".format(i,c));
else:
    print("The graph is not 3-colorable");

The graph is 3-colorable:
- Vertex 1 gets color 3
- Vertex 2 gets color 1
- Vertex 3 gets color 2
- Vertex 4 gets color 3
